# Estimating the Cost of Equity from Historical Price Data

We want to estimate the cost of equity for a company. We have historical data on its stock prices, as well as prices of a market portfolio. We will estimate the CAPM $\beta$, and then calculate the CAPM to determine the cost of equity.

:As a reminder, the CAPM formula is given by $$r_i = r_f + \beta (r_m - r_f) + \epsilon$$

## Load in Price Data

First let's load in the historical price data. We can use `pandas` to load the Excel file into Python. Ensure that the Excel workbook is in the same folder as your Jupyer notebook.



In [1]:
import pandas as pd

df = pd.read_excel('price data.xlsx')

In [2]:
df.head()  # print the first 5 rows

,Market Portfolio,Asset Price
0,1000.000000,100.000000
1,1178.861002,88.056952
2,1400.554023,71.554169
3,1476.669385,57.621757
4,1536.613924,47.839685


## Calculating Returns

The CAPM works with returns and not prices, so let's convert our prices to returns. Luckily the pandas method `pct_change` handles this for us.

In [3]:
returns = df.pct_change()
returns.head()

,Market Portfolio,Asset Price
0,NaN,NaN
1,0.178861,-0.119430
2,0.188057,-0.187410
3,0.054347,-0.194711
4,0.040594,-0.169764


The first values are missing (`NaN`) because we can't calculate a return off of a single number.

## Calculating the Market Risk Premium

We are ultimately going to be running a regression to determine $\beta$. We can think of a standard regression line as following the equation: $$y = a + bx$$ We can put the CAPM in this format if we assume $\epsilon$ is zero, then treat $r_i$ as $y$, $r_f$ as $a$, and $(r_m - r_f)$ as $x$. Therefore we need to calculate the market risk premium (MRP), $(r_m - r_f)$, to use in the regression. 

From the problem, the risk free rate is 3%. So just subtract that from the market returns to get the MRP.

In [4]:
risk_free = 0.03
returns['MRP'] = returns['Market Portfolio'] - risk_free
returns.head()

,Market Portfolio,Asset Price,MRP
0,NaN,NaN,NaN
1,0.178861,-0.119430,0.148861
2,0.188057,-0.187410,0.158057
3,0.054347,-0.194711,0.024347
4,0.040594,-0.169764,0.010594


## Calculating $\beta$

Now we are ready to run the regression of stock returns on the MRP. We can use `statsmodels` to run the OLS regression. We will also add a constant to the X variables, to have an intercept in the regression.

In [5]:
import statsmodels.api as sm

X = sm.add_constant(returns['MRP'])
y = returns['Asset Price']

model = sm.OLS(y, X)

MissingDataError: exog contains inf or nans

But you can see we ran into a problem `MissingDataError`. This is because we had those `NaN`s in the first row. We can remove these easily.

In [6]:
returns = returns.dropna()
returns.head()

,Market Portfolio,Asset Price,MRP
1,0.178861,-0.119430,0.148861
2,0.188057,-0.187410,0.158057
3,0.054347,-0.194711,0.024347
4,0.040594,-0.169764,0.010594
5,0.076758,0.227258,0.046758


Simply by using `.dropna()` we can remove those `NaN`s so we can run the regression. Let's try that again.

In [7]:
X = sm.add_constant(returns['MRP'])
y = returns['Asset Price']

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Asset Price   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.218
Method:                 Least Squares   F-statistic:                     28.53
Date:                Mon, 30 Mar 2020   Prob (F-statistic):           5.99e-07
Time:                        17:47:49   Log-Likelihood:                 13.883
No. Observations:                 100   AIC:                            -23.77
Df Residuals:                      98   BIC:                            -18.56
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0306      0.024      1.301      0.196      -0.016       0.077
MRP            0.8338      0.156      5.341      0.000       0.524       1.144
==============================================================================
Omnibus:                        1.312   Durbin-Watson:                   2.049
Prob(Omnibus):                  0.519   Jarque-Bera (JB):                1.114
Skew:                           0.010   Prob(JB):                        0.573
Kurtosis:                       2.483   Cond. No.                         7.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can see there is a 0.8338 coefficient on the MRP. This means our $\beta$ is 0.8338. We can extract that exact number as follows:

In [8]:
beta = results.params['MRP']
beta

0.8337833218355808

## Estimating the Market Return

Now we are only missing one component to plug into CAPM to get the cost of equity: the market return. A good way to estimate this is by taking an average of the historical returns. This can also be adjusted for expectations about the economy in the future (recession, etc.).

In [9]:
market_return = returns['Market Portfolio'].mean()
market_return

0.0944856620313094

## Estimating the Cost of Equity

Now we can plug everything into the CAPM formula to get the $r_i$ cost of equity. CAPM again: $$r_i = r_f + \beta (r_m - r_f) + \epsilon$$

In [10]:
cost_of_equity = risk_free + beta * (market_return - risk_free)
print(f'The cost of equity is {cost_of_equity:.2%}.')

The cost of equity is 8.38%.


## The Exercise in Excel

All the steps of the exercise are the same in Excel. The only difference is the functions/process to run each step. For calculating returns, a simple formula of $(new - old)/old$ can be calculated for one cell and dragged to get all the returns. 3% can be subtracted from the market returns and dragged down to yield the MRPs. The regression can be run by enabling the Data Analysis Toolpak add-in and following the prompts. 